In [1]:
# verwerken van AMR data export tot een lineaire tijdsreeks
# tijdszone 100 = UTC+1
# let op, de start en stop tijden worden aangepast bij de overgangen van zomer naar wintertijd en omgekeerd
# in de conversie zetten we alles om naar UTC+1 (dus in de zomer staat alles één uur verschoven)
# dit zullen we ook doen bij de injectie en productie data die uit NPS komt
import pandas as pd

# selectie van de date range, type energie en de kwartierwaarden zelf (bij de overgang van zomer naar wintertijd zijn er 4 extra kwartieren, dus 100 in plaats van 96)
# de *-operator zorgt voor expliciete unpacking van de range(), deze kan niet als argument van usecols meegegeven worden
col_list = usecols=[0,1,2,9, *range(11, 111)]

# inlezen van de csv file met selectie van rijen en kolommen
df = pd.read_csv('AMR_REPORTING_EXPORT_kwartierwaarden.csv', sep=';', header=None, usecols=col_list, skiprows=4, decimal=',')

# selecteer alleen de rijen betreft actieve energie (label A+)
df = df.loc[df[9] == 'A+']

# verwijderen van de kolom met 'A+' en reset index
df.drop(9, axis='columns', inplace=True)
df = df.reset_index()
del df['index']

# Combine the date and time columns into a single datetime string
df['start'] = df[0].astype(str) + ' ' + df[1].astype(str)

# Correctly format the date strings
# If the date string is 7 characters long, pad it with a leading zero
df['start'] = df['start'].apply(lambda x: '0' + x if len(x.split()[0]) == 7 else x)

# Correctly format the hour strings
# If the hour string is 1 character long, pad it with a leading zero
df['start'] = df['start'].apply(lambda x: x.split()[0] + ' 0' + x.split()[1] if len(x.split()[1].split(':')[0]) == 1 else x)

# Convert the datetime string to a datetime object
df['start'] = pd.to_datetime(df['start'], format='%d%m%Y %H:%M')

# You can now drop the original date and time columns
df.drop([0, 1], axis=1, inplace=True)

# Set the new datetime column as the index of the DataFrame
df.set_index('start', inplace=True)

# Omzetten van de 2de kolom naar een datetime object (makkerlijker: geen zero padding nodig)
df = df.rename(columns={2: 'einde'})
df['einde'] = pd.to_datetime(df['einde'], format='%d%m%Y %H:%M')

# Create a dictionary where the key is the old column name and the value is the new column name
rename_dict = {i: i-10 for i in range(11, 111)}

# Use the rename method to rename the columns
df.rename(columns=rename_dict, inplace=True)

# Reset the index (maakt van de datetime index de eerste kolom)
df = df.reset_index()

df.head(200)

# alle kolommen onder elkaar zetten
#df = df.stack()
# datetime met 15-min resolutie als index instellen (startdatum manueel opgeven)
#df.index = pd.date_range(start='2021-01-01', periods=len(df), freq='15min')
# opslaan van de afname data voor eventuele naverwerking in excel
#df.to_csv('eeklo_afname_2022.csv', index=True, header=False, sep=';')

,start,einde,1,2,3,4,5,6,7,8,...,91,92,93,94,95,96,97,98,99,100
0,2021-01-01 00:00:00,2021-01-02 00:00:00,13.90,14.45,14.35,13.85,13.25,13.15,13.50,13.10,...,15.30,15.15,14.15,14.65,14.35,13.95,NaN,NaN,NaN,NaN
1,2021-01-02 00:00:00,2021-01-03 00:00:00,15.00,14.60,14.00,14.20,13.05,12.90,13.35,13.30,...,13.95,14.10,13.25,13.05,13.40,12.55,NaN,NaN,NaN,NaN
2,2021-01-03 00:00:00,2021-01-04 00:00:00,13.30,13.20,13.15,12.65,13.20,12.90,13.55,13.25,...,14.90,14.10,13.70,13.65,12.95,13.10,NaN,NaN,NaN,NaN
3,2021-01-04 00:00:00,2021-01-05 00:00:00,13.95,13.10,13.50,13.55,13.75,13.40,13.55,12.95,...,15.40,15.15,14.50,14.25,14.50,14.50,NaN,NaN,NaN,NaN
4,2021-01-05 00:00:00,2021-01-06 00:00:00,14.65,14.55,14.55,14.90,14.05,14.35,14.65,14.10,...,16.80,16.40,15.75,16.15,15.85,16.45,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021-07-14 23:00:00,2021-07-15 23:00:00,16.55,17.10,16.50,17.10,15.50,15.95,15.25,15.25,...,16.85,17.95,16.95,18.25,16.35,16.35,NaN,NaN,NaN,NaN
196,2021-07-15 23:00:00,2021-07-16 23:00:00,15.10,16.10,15.25,17.00,16.00,16.25,15.75,16.50,...,17.95,18.60,17.45,17.95,17.45,17.20,NaN,NaN,NaN,NaN
197,2021-07-16 23:00:00,2021-07-17 23:00:00,16.20,16.65,16.25,17.95,16.85,16.95,16.70,16.90,...,15.85,15.65,15.30,15.35,15.75,16.55,NaN,NaN,NaN,NaN
198,2021-07-17 23:00:00,2021-07-18 23:00:00,16.70,17.40,17.20,17.80,17.05,17.15,15.35,15.35,...,16.80,15.90,15.65,16.10,15.60,16.40,NaN,NaN,NaN,NaN


In [2]:
# Omzetten naar één time-series (terminologie = smelten)

# Melt the dataframe to long format
df_melt = df.melt(id_vars=['start', 'einde'], var_name='time', value_name='value')

# Drop NaN values
df_melt = df_melt.dropna(subset=['value'])

# Convert the 'time' column to timedelta and add it to 'start'
df_melt['time'] = pd.to_timedelta(df_melt['time'].astype(int) * 15, unit='m')
df_melt['datetime'] = df_melt['start'] + df_melt['time']

# Drop unnecessary columns and set 'datetime' as index
df_melt = df_melt.drop(columns=['start', 'einde', 'time'])
df_melt = df_melt.set_index('datetime')

# Sort the dataframe by the index
df_melt = df_melt.sort_index()

# Find out which day has not 96 quarters of data
df_counts = df_melt.resample('D').count()

# Get the dates which do not have 96 values
dates_not_96 = df_counts[df_counts['value'] != 96].index.date

# Filter the rows in df_melt where the date is in dates_not_96
df_not_96 = df_melt[df_melt.index.normalize().isin(dates_not_96)]

# Filter the rows in df_melt where the date is not in dates_not_96
df_96 = df_melt[~df_melt.index.normalize().isin(dates_not_96)]

In [3]:
# Onderzoeken waar de afwijkende waarden zitten om verder te onderzoeken

# Make a copy of the sliced dataframe
df_not_96_copy = df_not_96.copy()

# convert to a wider format
df_not_96_copy['time'] = df_not_96_copy.index.time
df_not_96_copy = df_not_96_copy.reset_index()
df_not_96_copy['datetime'] = pd.to_datetime(df_not_96_copy['datetime'])
df_not_96_pivot = df_not_96_copy.pivot_table(index=df_not_96_copy['datetime'].dt.date, columns='time', values='value')

df_not_96_pivot.to_csv('eeklo_afwijkend.csv', index=True, header=True, sep=';')

In [ ]:
# Corrigeren van afwijkende waarden

In [ ]:
# Samenvoegen van gecorrigeerde data en de data zonder afwijkingen